In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import ModelCheckpoint

In [2]:
df = pd.read_csv('/content/ETHUSD_5m.csv' , names =['a' ,'unix','date','symbol','open','high','low','close','Volume_USD','Volume_crypto', 'k'] )
df = df.drop(['a' , 'k' , 'unix' ,'symbol' ] , axis = 1)
df.head()

,date,open,high,low,close,Volume_USD,Volume_crypto
0,2018-06-01 00:06:00,579.08,579.56,577.52,579.48,6.880892,3985.483745
1,2018-06-01 00:11:00,579.18,581.79,578.75,579.43,1196.594957,693651.637925
2,2018-06-01 00:16:00,579.32,580.72,578.95,579.30,1217.288639,705647.643435
3,2018-06-01 00:21:00,579.52,580.05,578.69,579.52,1268.806168,735517.111700
4,2018-06-01 00:26:00,580.60,581.08,579.83,581.08,1326.576505,769063.564014


In [3]:
df = df.dropna()
df = df.loc[:, df.columns != 'date']
df = df.values

In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df = scaler.fit_transform(df)

In [5]:
cand = 30
X = []
y = []
counter = 0 
for i in range(len(df)):
    temp = []
    counter = counter + 1
    try:
      for j in range(cand):
          temp.append(df[i+j])
      X.append(temp)
      y.append(df[i+cand][3])
    except:
      pass
          

In [6]:
X = np.array(X)
y = np.array(y)
X = X[:len(y)]

In [7]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.20, random_state=42)

In [12]:
def baseline_model():
    model = Sequential()
    model.add(LSTM(32, input_shape=((cand, 6)), return_sequences=True))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(LSTM(16, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(LSTM(8))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['mean_absolute_error'])
    return model

estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=32)


<ipython-input-12-bf6124912de4>:17: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=32)


In [13]:
mch = ModelCheckpoint('fianl-model.h5' , monitor='val_mean_absolute_error' , mode ='min' , save_best_only=True)

In [ ]:
history = estimator.fit(train_X,train_y , validation_data = (test_X , test_y) , callbacks = [mch])

Epoch 1/100
7701/7701 [==============================] - 271s 35ms/step - loss: 0.0065 - mean_absolute_error: 0.0403 - val_loss: 2.8792e-04 - val_mean_absolute_error: 0.0120
Epoch 2/100
7701/7701 [==============================] - 270s 35ms/step - loss: 8.0134e-04 - mean_absolute_error: 0.0160 - val_loss: 7.7395e-04 - val_mean_absolute_error: 0.0152
Epoch 3/100
7701/7701 [==============================] - 261s 34ms/step - loss: 5.2826e-04 - mean_absolute_error: 0.0129 - val_loss: 0.0011 - val_mean_absolute_error: 0.0187
Epoch 4/100
7701/7701 [==============================] - 264s 34ms/step - loss: 4.9685e-04 - mean_absolute_error: 0.0127 - val_loss: 0.0016 - val_mean_absolute_error: 0.0208
Epoch 5/100
7701/7701 [==============================] - 261s 34ms/step - loss: 3.6689e-04 - mean_absolute_error: 0.0107 - val_loss: 0.0013 - val_mean_absolute_error: 0.0181
Epoch 6/100
7701/7701 [==============================] - 255s 33ms/step - loss: 3.3817e-04 - mean_absolute_error: 0.0102 - val